# Init

In [ ]:
%gui qt
%matplotlib qt

import os

import numpy as np
from matplotlib import pyplot as plt
from importlib import reload

import qcodes as qc
from qcodes import ParamSpec, new_data_set
from qcodes.dataset.database import initialise_database
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import load_or_create_experiment

from plottr.data.datadict import DataDict
from plottr.data.qcodes_dataset import datadict_from_path_and_run_id
from plottr.apps.tools import make_sequential_flowchart
from plottr.apps import inspectr

# Make 2D softsweep data

In [ ]:
DBPATH = './test_qc_saveandload.db'

## define sweep

In [ ]:
qc.config.core.db_location = DBPATH
initialise_database()
exp = load_or_create_experiment('2d_softsweep', sample_name='no sample')

x = np.linspace(0, 1., 11)
y = np.linspace(0, 1., 11)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.random.rand(*xx.shape)

def get_2dsoftsweep_results():
    for x, y, z in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)):
        yield dict(x=x, y=y, z=z)

## dummy measurement and retrieve as ddict

In [ ]:
ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                  specs=[ParamSpec('x', 'numeric', unit='A'),
                         ParamSpec('y', 'numeric', unit='B'),
                         ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

results = get_2dsoftsweep_results()
for r in results:
    ds.add_result(r)
ds.mark_complete()

run_id = ds.run_id
ddict = datadict_from_path_and_run_id(DBPATH, run_id)

# Continously add data to a database

In [ ]:
DBPATH = './test_keep_adding_data.db'

## Define data sweep

In [ ]:
# def specs_from_ddict(ddict):
#     specs = []
#     for ax in ddict.axes():
    
# def ddict_to_results(ddict, ds_name=None, exp_id=None):        

In [ ]:
qc.config.core.db_location = DBPATH
initialise_database()
exp = load_or_create_experiment('2d_softsweep', sample_name='no sample')

x = np.linspace(0, 1., 11)
y = np.linspace(0, 1., 11)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.random.rand(*xx.shape)

def get_2dsoftsweep_results():
    for x, y, z in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)):
        yield dict(x=x, y=y, z=z)

## create data set

In [ ]:
ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                  specs=[ParamSpec('x', 'numeric', unit='A'),
                         ParamSpec('y', 'numeric', unit='B'),
                         ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

results = get_2dsoftsweep_results()

## add data row by row

In [ ]:
try:
    res = next(results)
    ds.add_result(res)
except StopIteration:
    ds.mark_complete()
    
print('number of results:', ds.number_of_results)
print('completed:', ds.completed)

In [ ]:
win = inspectr.inspectr(DBPATH)
win.show()